### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import test as t

import ratings as ratings

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('./data/original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('./data/original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

#### 1. Take a Look At The Data

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [28]:
# Use this cell to find the following information:
# number of movies
n_videos = movies.shape[0]
# number of ratings
n_reviews = reviews.shape[0]
# number of different genres

def _get_genre(_ds):
    _set = set()
    for v in _ds:
        if isinstance(v, str):
            for s in v.split('|'):
                _set.add(s)
        else:
            _set.add(v)

    return _set

n_genres = len(_get_genre(movies['genre'].values))
# number of unique users
n_users = len(reviews['user_id'].unique())
# number of missing ratings
n_missing_reviews = len(reviews['rating'].isnull())
# the `average`, `min`, and `max` ratings given
r_average, r_min, r_max = reviews['rating'].mean(), reviews['rating'].min(), reviews['rating'].max()




#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns.

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [37]:
import re

def _extract_date(df):

    def _extract_from_regex(movie):
        match = re.match(r'.*([1-3][0-9]{3})', movie)
        if match is not None:
            # Then it found a match!
            return match.group(1)

        return np.nan

    return df['movie'].apply(_extract_from_regex).astype(int)

def _bin_year(df):
    bins = [('1800s', lambda v: 1 if 1800 <= v < 1900 else 0),
            ('1900s', lambda v: 1 if 1900 <= v < 2000 else 0),
            ('2000s', lambda v: 1 if 2000 <= v else 0),

            ]

    for bin_name, bin_func in bins:
        df[bin_name] = df['year'].apply(bin_func)

    return df

def _bin_genres(_df):
    def _get_genre(_ds):
        _set = set()
        for v in _ds:
            if isinstance(v, str):
                for s in v.split('|'):
                    _set.add(s)
            else:
                _set.add(v)

        return _set

    for v in _get_genre(_df['genre']):
        _df[v] = _df['genre'].apply(lambda c: 1 if isinstance(c, str) and v in c.split('|') else 0)



movies['year'] = _extract_date(movies)

_bin_year(movies)
_bin_genres(movies)

movies

,movie_id,movie,genre,year,1800s,1900s,2000s,NaN,Sport,Documentary,...,Music,Western,Comedy,War,Adventure,News,History,Fantasy,Film-Noir,Crime
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35474,12036052,Elephant (2020),Documentary|Adventure|Biography|Family,2020,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
35475,12058882,Coachella: 20 Years in the Desert (2020),Documentary|Music,2020,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
35476,12078990,Money Heist: The Phenomenon (2020),Documentary,2020,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
35477,12131262,Out of Shadows (2020),Documentary,2020,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror
...,...,...,...
35474,12036052,Elephant (2020),Documentary|Adventure|Biography|Family
35475,12058882,Coachella: 20 Years in the Desert (2020),Documentary|Music
35476,12078990,Money Heist: The Phenomenon (2020),Documentary
35477,12131262,Out of Shadows (2020),Documentary
